# Numpy basics

As a reminder, objects in Python (including numbers!) are abstract and dynamic:

   * you don't know where they are in memory (pointer address); could be anywhere
   * you don't know how they're laid out in bytes
   * data types, member data/functions, etc. are checked at the last possible moment

And so they are slow.

In [44]:
import random
data = []
for i in range(1000000):
    data.append(random.gauss(0, 1))

In [45]:
%%timeit
data2 = []
for x in data:
    data2.append(x**2)

10 loops, best of 3: 137 ms per loop


But Numpy isn't.

In [46]:
import numpy
data = numpy.random.normal(0, 1, 1000000)

In [47]:
%%timeit
data2 = data**2

The slowest run took 4.97 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 595 µs per loop


How does it work? A Numpy array is everything a list of Python objects is not:

   * the data are known to be contiguous in memory (sequential access is important!)
   * you can directly access their bytes
   * the data type of an array is specified once for the whole array
   * *bonus:* most methods benefit from hardware vectorization
   * *bonus:* all methods release Python's interpreter lock, so parallel threads can run at the same time
   * *bonus:* without a type pointer for each object, numbers use less memory

Numpy forces a different order of operations: instead of processing a table of data one event at a time, it only helps if you process one operation (for all events) at a time.

In [59]:
px = numpy.random.normal(0, 30, 100000)
py = numpy.random.normal(0, 30, 100000)
pz = numpy.random.normal(0, 300, 100000)

Instead of

In [60]:
%%timeit
p = numpy.empty(100000)
for i in range(len(p)):                                   # for each px[i], py[i], pz[i]
    p[i] = numpy.sqrt(px[i]**2 + py[i]**2 + pz[i]**2)     # compute p[i]

1 loop, best of 3: 255 ms per loop


do

In [61]:
%%timeit
p = numpy.sqrt(px**2 + py**2 + pz**2)       # compute all px**2, then all py**2, then all pz**2, then sum all, then sqrt all

1000 loops, best of 3: 540 µs per loop


Normal math functions are *scalar* (binary operators like `+` or functions from `import math`),

Numpy math functions are *vectorized.* Given equal-length arrays as input, they return the same length array as output, performing all loops in compiled C or even vectorized across a CPU. (Some implementations perform the work in parallel or on a GPU, but not the default one.)

In [69]:
small_array = numpy.array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [70]:
small_array**2

array([ 1,  4,  9, 16, 25, 36, 49, 64, 81])

In [71]:
numpy.sqrt(small_array)

array([1.        , 1.41421356, 1.73205081, 2.        , 2.23606798,
       2.44948974, 2.64575131, 2.82842712, 3.        ])

In [72]:
import math
math.sqrt(small_array)

TypeError: only size-1 arrays can be converted to Python scalars

They are raw bytes; you can do whatever you want with them.

In [74]:
asbytes = small_array.view(numpy.uint8)
asbytes

array([1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0,
       0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 9, 0,
       0, 0, 0, 0, 0, 0], dtype=uint8)

In [75]:
asbytes[10] = 123
small_array

array([      1, 8060930,       3,       4,       5,       6,       7,
             8,       9])

They may have arbitrarily many dimensions. Changing the dimensions (in a way that keeps the total number of items constant) *does not change the underlying data.*

In [80]:
small_array.reshape(3, 3)

array([[      1, 8060930,       3],
       [      4,       5,       6],
       [      7,       8,       9]])

The columns can be named, making it easy to swap array-of-structs with struct-of-arrays.

In [81]:
recarray = small_array.view([("one", int), ("two", int), ("three", int)])
recarray

array([(1, 8060930, 3), (4,       5, 6), (7,       8, 9)],
      dtype=[('one', '<i8'), ('two', '<i8'), ('three', '<i8')])

In [82]:
recarray["one"]

array([1, 4, 7])

In [83]:
recarray["two"]

array([8060930,       5,       8])

In [84]:
recarray["three"]

array([3, 6, 9])

slicing, fancy indexing, "base" (view vs copy), getting the pointer, making vectorized functions in ROOT